In [70]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

from sklearn.utils import resample
from sklearn.model_selection import train_test_split

from method.MTS_base import MTSClassifier
from dataload.data_load import data_load
from func.my_func import make_result_df

In [71]:
ex_name = input('実験名は?')

n_experiment = 10
data_list = [
    'yeast', 
    'wine', 
    'abalone', 
    'car',
    'cancer', 
    'letter'
    ]

print('MTS開始！')

for data in data_list:
    print(data)
    result_df = pd.DataFrame(
                        columns=['AUC', 'accuracy', 'recall', 'Specificity', 'precision', 'gmeans', 'RS'],
                        index=range(n_experiment))
    for m in tqdm(range(n_experiment)):
        
        X, y = data_load(data)

        # バギング側の話
        # ブートストラップサンプリングの個数
        n = 10
        seed = random.randint(0, n)

        # 使用する7つの変数をランダムに取得する
        # バギングをする際はそれぞれのサブサンプルで7つの変数を選択する．
        random.seed(seed)
        random_s = random.sample(list(X.columns), len(X.columns) if len(X.columns) < 7 else 7)
        X = X[random_s]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        clf = MTSClassifier(add_weight=False)
        clf.fit(X_train, y_train)
        y_proba = clf.predict_proba(X_test)

        clf.determine_threshold(X_train, y_train)
        y_pred = clf.predict(X_test)

        result_df = make_result_df(result_df, y_test, y_pred, y_proba, m)
        
    result_df.to_csv(f'../data/output/{ex_name}_MTS_{data}_result.csv')

MTS開始！
yeast


  0%|          | 0/10 [00:00<?, ?it/s]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [74]:
clf.predict_proba(X_test)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [62]:
if len(clf.select_columns) > 1:
    Z = clf.reduced_model_scaler.transform(X[clf.select_columns])
    if clf.add_weight:
        Weighted_Z = Z * clf.select_columns_weight
        # MD = cal_MD(Weighted_Z, clf.reduced_model_inv_C)
    # else:
        # MD = cal_MD(Z, clf.reduced_model_inv_C)
# 変数が一つしか選択されなかった場合はその変数を正常データの平均と標準偏差で標準化してそれの二乗を異常値とする
else:
    MD = ((X[clf.select_columns] - clf.reduced_model_mean) / clf.reduced_model_std) ** 2

In [66]:
Z

array([[ 0.74427584, -0.51836484, -0.87118324,  0.19105732, -1.0807469 ],
       [ 0.89144903, -0.51836484, -1.8921011 ,  0.71897887,  0.03178667],
       [ 1.03862222, -0.51836484, -0.4628161 ,  0.27904425, -0.92181353],
       ...,
       [-0.87462924, -0.51836484, -0.25863253, -0.16089038, -0.6039468 ],
       [ 2.6575273 ,  2.76277429, -1.8921011 , -1.65666811, -0.84234685],
       [ 1.77448817, -0.51836484, -0.39475491, -0.42485115, -1.0807469 ]])

In [10]:
len('') != 0

False

In [16]:
clf.add_weight

False

In [26]:
clf.select_columns_weight.all() >= 0

True

In [46]:
np.ndarray([0,0]).any()

False

In [47]:
type(clf.select_columns_weight) == np.ndarray

True